In [ ]:
# ---------------------------------
# 문제정의
# ---------------------------------
# 주어진 제품 및 매장 특성들을(X값,컬럼) 기반으로 판매 금액(Item_Outlet_Sales)을 예측하기

In [ ]:
# ---------------------------------
# 라이브러리 가져오기
# ---------------------------------

In [1]:
import pandas as pd
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch4/train.csv")
# test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part2/ch4/test.csv")
train

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,NCR06,12.500,Low Fat,0.006760,Household,42.8112,OUT013,1987,High,Tier 3,Supermarket Type1,639.1680
1,FDW11,12.600,Low Fat,0.048741,Breads,60.4194,OUT013,1987,High,Tier 3,Supermarket Type1,990.7104
2,FDH32,12.800,Low Fat,0.075997,Fruits and Vegetables,97.1410,OUT013,1987,High,Tier 3,Supermarket Type1,2799.6890
3,FDL52,6.635,Regular,0.046351,Frozen Foods,37.4506,OUT017,2007,NaN,Tier 2,Supermarket Type1,1176.4686
4,FDO09,13.500,Regular,0.125170,Snack Foods,261.4910,OUT013,1987,High,Tier 3,Supermarket Type1,3418.8830
...,...,...,...,...,...,...,...,...,...,...,...,...
6813,NCC54,17.750,Low Fat,0.097863,Health and Hygiene,239.3196,OUT049,1999,Medium,Tier 1,Supermarket Type1,3615.2940
6814,NCF19,13.000,Low Fat,0.035252,Household,48.0034,OUT018,2009,Medium,Tier 3,Supermarket Type2,923.4646
6815,FDU11,4.785,Low Fat,0.092576,Breads,121.0098,OUT035,2004,Small,Tier 2,Supermarket Type1,2530.7058
6816,FDK14,6.980,Low Fat,0.041189,Canned,82.8934,OUT045,2002,NaN,Tier 2,Supermarket Type1,818.9340


In [ ]:
# ---------------------------------
# EDA
# ---------------------------------

In [ ]:
# train.head()
# train.tail()
train.sample(5)
train.shape, test.shape
train.info()
test.info()
train.describe()
test.describe()
train.describe(include='O')
train.isnull().sum()
test.isnull().sum()
train.isnull().sum().sum()
test.isnull().sum().sum()

In [ ]:
# ---------------------------------
# 데이터 전처리
# ---------------------------------

In [2]:
# 결측값 채우기(무게 : 최소값 , 사이즈 : 최빈값)

train['Item_Weight'] = train['Item_Weight'].fillna(train['Item_Weight'].min())
train['Outlet_Size'] = train['Outlet_Size'].fillna(train['Outlet_Size'].mode()[0])

test['Item_Weight'] = test['Item_Weight'].fillna(test['Item_Weight'].min())
test['Outlet_Size'] = test['Outlet_Size'].fillna(test['Outlet_Size'].mode()[0])


In [3]:
# 컬럼제거
train.drop('Item_Identifier',axis=1, inplace=True)
test.drop('Item_Identifier',axis=1, inplace=True)

In [ ]:
#----------------------------------------------
# ★ OLS - 선형회귀에서만 사용할 것(분류는 x)
#----------------------------------------------
from statsmodels.formula.api import ols

# y 항목컬럼명 저장
y_col = "Item_Outlet_Sales"

# y를 뺀 x값 후보들(수치형,범주형 다 포함됨)
X_cols = [c for c in train.columns if c != y_col]
X_cols

# # formula 생성
formula = f"{y_col} ~ " + " + ".join(X_cols)
formula

# # OLS 모델 적용
model = ols(formula, data=train).fit()

# # 확인 
print(model.summary())


In [ ]:
#----------------------------------------------
# [★] OLS - Pvalue 값 높은값(0.05이상) 만 제거 (이상이면 y값과 상관관계가 낮음)
#----------------------------------------------
# PValue 0.05 이상인 컬럼 확인
list = model.pvalues[model.pvalues > 0.05].index.tolist()
list

# 컬럼삭제
train = train.drop('Item_Fat_Content',axis=1)
train = train.drop('Item_Type',axis=1)
train = train.drop('Outlet_Identifier',axis=1) 


test = test.drop('Item_Fat_Content',axis=1)
test = test.drop('Item_Type',axis=1)
test = test.drop('Outlet_Identifier',axis=1)

In [ ]:
#----------------------------------------------
# OLS 다중공선성 의심(다중공선성 : 설명 변수들(독립변수,X1,X2..) 사이에 서로 강한 상관관계존재) 
#----------------------------------------------
# Cond. No.                     1.44e+19

# 발생 원인
# 범주형 독립변수 간(X1,X2)의 중복된 의미가 있을때 -> 범주형-수치형 검정(카이제곱)
# 연속된 수치값(ex 날짜정보 ) 가 포함되어있을때 -> 컬럼 삭제
# 수치형 독립변수(X1,X2)들간의 변수 범위 차이가 클때 -> 스케일링

# Cond. No.                    
# < 30                | 문제 없음                    
# 30 ~ 100            | 약한 다중공선성 가능성            
# 100 ~ 1,000         | 다중공선성 의심              
# **> 1,000**         | **심각한 다중공선성 가능성 있음**   
# **> 10⁵ (100,000)** | **수치적으로 매우 불안정 — 경고 수준** 


# 다중공선성 원인 변수 찿기(VIF) - 다중공선성 의 정도를 수치로 측정한 지표
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd

X_cols = [c for c in train.columns if c != y_col]

X = train[X_cols].select_dtypes(include=['float', 'int'])
X
vif = pd.DataFrame()
vif["Feature"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif

# VIF > 10 → 다중공선성 심함
# VIF > 5 → 주의 필요
# → Outlet_Establishment_Year

# 해결책
# 수치형 변수 스케일링 - StandardScaler 또는 MinMaxScaler
# 다중공선성 높은 변수 제거  -  VIF 10 이상인 변수 제거
# Ridge/Lasso 회귀로 대체  - 정규화로 공선성 완화

In [4]:
#----------------------------------------------
# [★] vif > 10 컬럼삭제
#----------------------------------------------
train = train.drop('Outlet_Establishment_Year',axis=1)
test = test.drop('Outlet_Establishment_Year',axis=1)


In [ ]:
# ---------------------------------------------------------
# [★] OLS 수치형 변수 스케일링  (StandardScaler : -1 ~ 1 )
# ---------------------------------------------------------
from sklearn.preprocessing import StandardScaler

# 1. 수치형 컬럼 자동 선택 (object 제외)
num_cols = train.select_dtypes(exclude=['object']).columns
# 2. 스케일러 선언
scaler = StandardScaler()
# 3. 수치형 컬럼만 스케일링
train[num_cols] = scaler.fit_transform(train[num_cols])
# 4. 결과 확인
print("스케일링 완료된 수치형 컬럼들:")
print(num_cols.tolist())
print(train.head())

In [ ]:
train

In [ ]:
# 이상치 - 생략

In [5]:
# y값 분할 
target = train.pop('Item_Outlet_Sales')
target

0        639.1680
1        990.7104
2       2799.6890
3       1176.4686
4       3418.8830
          ...    
6813    3615.2940
6814     923.4646
6815    2530.7058
6816     818.9340
6817    1973.4312
Name: Item_Outlet_Sales, Length: 6818, dtype: float64

In [ ]:
# 레이블인코딩

In [6]:
from sklearn.preprocessing import LabelEncoder

cols = train.columns[train.dtypes == object]

for col in cols:
    le = LabelEncoder()
    le.fit(pd.concat([train[col],test[col]] ,axis=0)) # 행합치기
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])

In [7]:
test

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,17.250,1,0.073251,3,76.4986,2,1,1,1
1,6.360,1,0.120544,6,45.8060,8,2,0,1
2,10.695,2,0.128247,4,121.8440,3,1,2,2
3,7.405,1,0.015296,3,89.7146,9,1,0,1
4,10.500,2,0.142760,1,159.3578,9,1,0,1
...,...,...,...,...,...,...,...,...,...
1700,4.555,1,0.097769,9,142.4496,4,2,0,0
1701,4.555,2,0.153457,4,264.0910,4,2,0,0
1702,20.750,2,0.129864,12,240.1880,3,1,2,2
1703,14.650,1,0.171282,7,54.0614,2,1,1,1


In [ ]:
# ---------------------------------
# 검증 데이터 분할 train - val (8,2)
# ---------------------------------

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    train,
    target,
    test_size=0.2,
    random_state=0
)
X_train.shape, X_val.shape, y_train.shape, y_val.shape




((5454, 9), (1364, 9), (5454,), (1364,))

In [ ]:
# ---------------------------------
# 학습 및 평가(MSE, MAE , R2..)
# ---------------------------------

In [ ]:
# LIGHTGBM 학습

In [20]:
# LIGHTGBM 학습
import lightgbm as lgb
model = lgb.LGBMRegressor(random_state=0, verbose=-1)
model.fit(X_train, y_train)
y_pred_2 = model.predict(X_val)


In [21]:
# ------------------------------------------------------------
# 회귀 모델 성능 평가 지표 불러오기
# ------------------------------------------------------------
from sklearn.metrics import mean_squared_error   # MSE(평균제곱오차)
from sklearn.metrics import mean_absolute_error  # MAE(평균절대오차)
from sklearn.metrics import r2_score             # R2(결정계수)

# ------------------------------------------------------------
# MSE (Mean Squared Error)
# 실제값(y_val)과 예측값(y_pred_2)의 제곱 오차 평균
# ------------------------------------------------------------
lgm_mse = mean_squared_error(y_val, y_pred_2)

# ------------------------------------------------------------
# MAE (Mean Absolute Error)
# 실제값과 예측값의 절대 오차 평균
# ------------------------------------------------------------
lgm_mae = mean_absolute_error(y_val, y_pred_2)

# ------------------------------------------------------------
# RMSE (Root Mean Squared Error)
# MSE의 제곱근으로 계산 (또는 squared=False 옵션 사용 가능)
# ------------------------------------------------------------
lgm_rmse = mean_squared_error(y_val, y_pred_2, squared=False)

# ------------------------------------------------------------
# R² (결정계수)
# 모델이 타깃 변수의 분산을 얼마나 설명하는지 (1에 가까울수록 좋음)
# ------------------------------------------------------------
lgm_r2 = r2_score(y_val, y_pred_2)

# ------------------------------------------------------------
# 결과 출력
# ------------------------------------------------------------
print("MSE  :", lgm_mse)
print("MAE  :", lgm_mae)
print("RMSE :", lgm_rmse)
print("R2   :", lgm_r2)


MSE  : 1105701.377973892
MAE  : 737.1335186972819
RMSE : 1051.5233606410711
R2   : 0.5740828013539143


# ----------------------------------
## 상관계수(x후보들 - y값) 낮은 컬럼 제거
# ----------------------------------

In [22]:
# 임계값 조절

TARGET_CORR_TH = 0.03 # 상관계수 임계값

#숫자형 기준으로 상관계수 계산
df_corr = train.copy()
df_corr['__target__'] = y_train
corr_with_target = df_corr.corr(numeric_only=True)['__target__'].drop('__target__')
# corr_with_target


#유지할 컬럼(= 타깃과의 절대 상관이 임계값 이상인 컬럼만 추출)
keep_features = corr_with_target[ corr_with_target.abs() >= TARGET_CORR_TH ].index.tolist()
keep_features

#제거할 컬럼
problem_features = corr_with_target[ corr_with_target.abs() < TARGET_CORR_TH ].index.tolist()
# problem_features


In [23]:
# ----------------------------------
# 컬럼제거
# ----------------------------------
train.drop('Item_Fat_Content',axis=1,inplace=True)
train.drop('Item_Type',axis=1,inplace=True)

test.drop('Item_Fat_Content',axis=1,inplace=True)
test.drop('Item_Type',axis=1,inplace=True)

KeyError: "['Item_Fat_Content'] not found in axis"

In [24]:
# ----------------------------------
# 확인
# ----------------------------------
train.shape, test.shape

((6818, 7), (1705, 7))

In [25]:
from sklearn.model_selection import train_test_split

X_train_re, X_val_re, y_train_re, y_val_re = train_test_split(
    train,
    target,
    test_size=0.2,
    random_state=0
)
X_train_re.shape, X_val_re.shape, y_train_re.shape, y_val_re.shape

((5454, 7), (1364, 7), (5454,), (1364,))

In [26]:
import lightgbm as lgb
model = lgb.LGBMRegressor(random_state=0, verbose=-1)
model.fit(X_train_re, y_train_re)
y_pred_2_re = model.predict(X_val_re)

In [27]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# ------------------------------------------------------------
# 성능 지표 계산
# ------------------------------------------------------------
lgm_mse_re = mean_squared_error(y_val_re, y_pred_2_re)               # MSE: 평균제곱오차
lgm_mae_re = mean_absolute_error(y_val_re, y_pred_2_re)              # MAE: 평균절대오차
lgm_rmse_re = mean_squared_error(y_val_re, y_pred_2_re, squared=False) # RMSE: 평균제곱근오차(원래 단위로 반환)
lgm_r2_re = r2_score(y_val_re, y_pred_2_re)                          # R²: 결정계수 (1에 가까울수록 좋음)

# ------------------------------------------------------------
# 결과 출력
# ------------------------------------------------------------
print(f"MSE  : {lgm_mse_re:,.4f}  # 오차 제곱의 평균 (낮을수록 좋음)")
print(f"MAE  : {lgm_mae_re:,.4f}  # 오차 절댓값 평균 (낮을수록 좋음)")
print(f"RMSE : {lgm_rmse_re:,.4f}  # MSE의 제곱근 (해석 쉬움, 낮을수록 좋음)")
print(f"R²   : {lgm_r2_re:.4f}     # 설명력 지표 (1에 가까울수록 좋음)")


MSE  : 1,095,162.2784  # 오차 제곱의 평균 (낮을수록 좋음)
MAE  : 732.9844  # 오차 절댓값 평균 (낮을수록 좋음)
RMSE : 1,046.5000  # MSE의 제곱근 (해석 쉬움, 낮을수록 좋음)
R²   : 0.5781     # 설명력 지표 (1에 가까울수록 좋음)
